# Linear / Ridge / Lasso Regression

## Mục tiêu: 
Huấn luyện các mô hình Linear / Ridge / Lasso cho tập dữ liệu huấn luyện về nhân khẩu học của các quốc gia trên thế giới.

## Bước 1 - Import các thư viện cần thiết

1.1. Import thư viện

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

1.2. Cấu hình thư mục

In [ ]:
os.makedirs("../model/1_linear_regression", exist_ok=True)

## Bước 2 - Đọc dữ liệu đã tiền xử lý
Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [19]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (dữ liệu đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 15)

5 dòng đầu tiên của tập train (dữ liệu đã được chuẩn hóa):


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Romania,ROU,2019,-0.100387,-0.593216,-1.101137,75.607317,-0.107887,0.079336,1.142611,0.621932,0.746494,-0.047183,-0.857182,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.685495,-0.500838,72.432195,-0.405891,0.241659,0.204355,0.586683,0.718065,-0.221970,1.767314,-0.189257
2,Angola,AGO,2008,-0.083899,0.283435,1.500859,55.281000,-0.470953,1.327705,-0.839543,-1.545881,-2.268636,-0.432669,0.305989,1.598947
3,Albania,ALB,2001,-0.222240,-0.619770,-1.349940,75.083000,-0.584364,0.930412,-0.835544,0.600783,-0.024271,-0.404910,0.108810,-0.104497
4,Central African Republic,CAF,2021,-0.206911,2.567341,0.236338,40.279000,-0.618434,-0.427187,-1.336901,-2.349558,-3.008624,-0.529029,1.729562,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (life_expectancy) khỏi các đặc trưng và loại bỏ các cột không cần thiết như country_name, country_code,
life_expectancy.

In [9]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 12

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. poverty_ratio
  4. pop_growth
  5. gdp_per_capita
  6. gdp_growth
  7. sanitation
  8. electricity
  9. water_access
  10. co2_emissions
  11. slum_population
  12. labor_force

Kích thước X_train: (3255, 12)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình Linear Regression 


4.1. Fit mô hình LinearRegression() trên tập train.

In [10]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_lin = linear_model.predict(X_train)

4.2. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [12]:
# Tính 3 loại loss: MAE, RMSE và R2
lin_rmse = np.sqrt(mean_squared_error(y_train, y_pred_lin))
lin_r2  = r2_score(y_train, y_pred_lin)
lin_mae = mean_absolute_error(y_train, y_pred_lin)

print("Model: LinearRegression:")
print(f"RMSE loss: {lin_rmse:.3f}")
print(f"MAE loss: {lin_mae:.3f}")
print(f"R2 loss: {lin_r2:.3f}")

# Lưu mô hình
joblib.dump(linear_model, "../model/1_linear_regression/linear_regression.pkl")

Model: LinearRegression:
RMSE loss: 4.154
MAE loss: 3.127
R2 loss: 0.771


['../model/1_linear_regression/linear_regression.pkl']

## Bước 5 - Xây dựng và huấn luyện mô hình Ridge
### Mục tiêu:
- Tối ưu hóa siêu tham số Ridge bằng GridSearchCV (k=5)
- So sánh MSE giữa các giá trị alpha [0.01, 0.05, 0.25, 0.5] để tìm ra siêu tham số tốt nhất.

5.1. Huấn luyện mô hình sử dụng phương pháp k-fold cross-validation (k=5) 

In [13]:
ridge_param_grid = {'alpha': [0.01, 0.05, 0.25, 0.5]}
ridge = Ridge(random_state=RANDOM_STATE)
ridge_cv = GridSearchCV(ridge, ridge_param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
ridge_cv.fit(X_train, y_train)
best_ridge_alpha = ridge_cv.best_params_['alpha']
print("Ridge best alpha (5-fold CV):", best_ridge_alpha)

Ridge best alpha (5-fold CV): 0.5


5.2. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [15]:
# Tính 3 loại loss: MAE, RMSE và R2
ridge_best = Ridge(alpha=best_ridge_alpha, random_state=RANDOM_STATE)
ridge_best.fit(X_train, y_train)
y_pred_ridge = ridge_best.predict(X_train)
ridge_rmse = np.sqrt(mean_squared_error(y_train, y_pred_ridge))
ridge_r2  = r2_score(y_train, y_pred_ridge)
ridge_mae = mean_absolute_error(y_train, y_pred_ridge)

print("Model: Ridge Regression:")
print(f"RMSE loss: {ridge_rmse:.3f}")
print(f"MAE loss: {ridge_mae:.3f}")
print(f"R2 loss: {ridge_r2:.3f}")

# Lưu mô hình
joblib.dump(ridge_best, f"../model/1_linear_regression/ridge_best_alpha_{best_ridge_alpha}.pkl")

Model: Ridge Regression:
RMSE loss: 4.154
MAE loss: 3.127
R2 loss: 0.771


['../model/1_linear_regression/ridge_best_alpha_0.5.pkl']

## Bước 6 - Xây dựng và huấn luyện mô hình Lasso
### Mục tiêu:
- Tối ưu hóa siêu tham số Lasso bằng GridSearchCV (k=5)
- So sánh MSE giữa các giá trị alpha [0.01, 0.05, 0.25, 0.5] để tìm ra siêu tham số tốt nhất.

6.1. Huấn luyện mô hình sử dụng phương pháp k-fold cross-validation (k=5) 

In [16]:
lasso_param_grid = {'alpha': [0.01, 0.05, 0.25, 0.5]}
lasso = Lasso(random_state=RANDOM_STATE, max_iter=5000)
lasso_cv = GridSearchCV(lasso, lasso_param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
lasso_cv.fit(X_train, y_train)
best_lasso_alpha = lasso_cv.best_params_['alpha']
print("Lasso best alpha (5-fold CV):", best_lasso_alpha)

Lasso best alpha (5-fold CV): 0.01


6.2. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [17]:
# Tính 3 loại loss: MAE, RMSE và R2
lasso_best = Lasso(alpha=best_lasso_alpha, random_state=RANDOM_STATE, max_iter=5000)
lasso_best.fit(X_train, y_train)
y_pred_lasso = lasso_best.predict(X_train)
lasso_rmse = np.sqrt(mean_squared_error(y_train, y_pred_lasso))
lasso_r2  = r2_score(y_train, y_pred_lasso)
lasso_mae = mean_absolute_error(y_train, y_pred_lasso)

print("Model: Lasso Regression:")
print(f"RMSE loss: {lasso_rmse:.3f}")
print(f"MAE loss: {lasso_mae:.3f}")
print(f"R2 loss: {lasso_r2:.3f}")

# Lưu mô hình
joblib.dump(lasso_best, f"../model/1_linear_regression/lasso_best_alpha_{best_lasso_alpha}.pkl")

Model: Lasso Regression:
RMSE loss: 4.154
MAE loss: 3.127
R2 loss: 0.771


['../model/1_linear_regression/lasso_best_alpha_0.01.pkl']

## Kết luận
Tổng kết quá trình huấn luyện 3 mô hình (Linear, Ridge, Lasso)

In [18]:
print("Hoàn tất: đã huấn luyện thành công các mô hình Linear/Ridge/Lasso!")
print("Dùng 5-fold CV để chọn alpha cho Ridge & Lasso")

Hoàn tất: đã huấn luyện thành công các mô hình Linear/Ridge/Lasso!
Dùng 5-fold CV để chọn alpha cho Ridge & Lasso
